## Convert Wflow staticmaps netcdf to raster files

In order to inspect or (manually) modify Wflow staticmaps it is convenient to export the maps to a raster format. Here we show how to read the model maps and save to a so-called mapstack (i.e.: a set of raster files with identical grid) using HydroMT.  

### Load dependencies

In [ ]:
import shutil
from os.path import join, exists
from os import listdir
import hydromt
from hydromt_wflow import WflowSbmModel

### Read wflow staticmaps

HydroMT provides an easy method to read the model schematization through the Model API.

In [ ]:
root = "wflow_piave_subbasin"
wflow = WflowSbmModel(root, mode="r")
ds = wflow.staticmaps.data  # here the staticmaps netcdf is loaded
print(ds)

### Write netcdf to mapstack

The raster module provides many raster GIS methods through the **raster** Dataset accessor. To write a Dataset into several raster files (eg one geotiff file per map), one line with code is sufficient using [raster.to_mapstack](https://deltares.github.io/hydromt/stable/_generated/hydromt.gis.Dataset.raster.to_mapstack.html). We only need to provide the output folder in which all raster files are saved. The default output format is *GeoTIFF*, but this can be changed with the `driver` argument.

In [ ]:
# Name of the folder where the tif files will be saved
updated_staticmaps = "updated_staticmaps"

# First remove the folder if it already exists
if exists(updated_staticmaps):
    shutil.rmtree(updated_staticmaps)

# Save the tif files
ds.raster.to_mapstack(updated_staticmaps)
# Print the content of the folder
listdir(updated_staticmaps)

Now the model files can easily be inspected and modified for example using QGIS (e.g. with the Serval plugin).

> **Note:** in QGIS, you can also visualize netcdf files but direct modification is not (yet) possible.

### Update a specific map using `setup_grid_from_raster`

Let's say you have updated (manually or not) the `soil_ksat_vertical.tif` static map and that you would like to include it in your model. For demonstration purpose, we will make a copy and name it `soil_ksat_vertical_updated.tif`.

In [ ]:
old_map = join(updated_staticmaps, "soil_ksat_vertical.tif")
updated_map = join(updated_staticmaps, "soil_ksat_vertical_updated.tif")
shutil.copyfile(old_map, updated_map)

You will need to find the `wflow variable` that corresponds to the map you updated, which in the case of `soil_ksat_vertical.tif` is `soil_surface_water__vertical_saturated_hydraulic_conductivity`. For information on the available wflow variables, see the [Wflow documentation](https://deltares.github.io/Wflow.jl/stable/model_docs/intro/)

And then you can simply use [setup_grid_from_raster](https://deltares.github.io/hydromt_wflow/stable/_generated/hydromt_wflow.WflowSbmModel.setup_grid_from_raster.html).

In the below example, we will use python to update our model. If you wish to update via command line, the steps are:

1. Create a data catalog entry for the maps in the "updated_staticmaps" folder
2. Prepare a hydromt config file with the steps: `setup_grid_from_raster`, `write_grid` and `write_config`
3. Call the `hydromt update` command line re-using the catalog and configuration file you created.

In [ ]:
# Instantiate a new WflowSbmModel object in read/write mode (r+) to be able to update it
wflow = WflowSbmModel(root, mode="r+")

# `KsatVer` corresponds to the `soil_surface_water__vertical_saturated_hydraulic_conductivity` variable
wflow_variable = "soil_surface_water__vertical_saturated_hydraulic_conductivity"
config_opt = wflow.get_config(f"input.static.{wflow_variable}")
print(f"Before setup_grid_from_raster: {config_opt}")

# Path to the (updated) `soil_ksat_vertical_updated.tif`
updated_map = join(updated_staticmaps, "soil_ksat_vertical_updated.tif")

# Update the model KsatVer map with setup_grid_from_raster
wflow.setup_grid_from_raster(
    raster_fn=updated_map, 
    reproject_method='nearest',
    variables=["soil_ksat_vertical_updated"], 
    wflow_variables=[wflow_variable],
    
)
config_opt = wflow.get_config(f"input.static.{wflow_variable}")
print(f"After setup_grid_from_raster: {config_opt}")

# Write the updated grid and config files to new files
wflow.write_grid(filename="staticmaps_updated.nc")
wflow.write_config(filename="wflow_sbm_updated.toml")
